제목 : 잠재의미분석(Latent Semantic Analysis, LSA) 실습 - Truncated SVD  
학습일 : 2022-07-05  
작성자 : [송유럽(Euro)](https://www.euroverse.dev/)  
관련 링크 : [딥러닝을 이용한 자연어처리 입문](https://wikidocs.net/24949)
  
--- 설명 ---  
Topic Modeling을 공부하다가 LSA에 대한 내용을 실습함.  
LSA는 토픽 모델링 분야에 아이디어를 제공한 알고리즘임.  
LSA의 단점을 개선하여 LDA라는 토픽 모델링에 적합한 알고리즘이 탄생함.   
이번 실습에서는 Truncated SVD를 통해 LSA를 이해해 보겠음.

--- 더 공부할 내용 ---  
특이값 분해(SVD) 제대로 공부하고 다시 보기  
np.diag()와 같이 넘파이 내용 공부하기

기존의 DTM이나 TF-IDF는 빈도 혹은 중요도를 고려할 뿐,  
단어의 의미는 전혀 고려하지 못한다는 단점을 갖고 있다.  
LSA는 기본적으로 DTM 혹은 TF-IDF 행렬에 Truncated SVD를 사용해  
차원을 축소시키고, 단어들의 잠재적인 의미를 끌어낸다는 아이디어이다.  

![img](../image/001.PNG)

In [1]:
import numpy as np
A = np.array([[0,0,0,1,0,1,1,0,0],[0,0,0,1,1,0,1,0,0],[0,1,1,0,2,0,0,0,0],[1,0,0,0,0,0,0,1,1]])
print('DTM의 크기(shape) :', np.shape(A))

DTM의 크기(shape) : (4, 9)


### Full SVD

In [ ]:
# Full SVD
U, s, VT = np.linalg.svd(A, full_matrices = True)

In [4]:
print("행렬 U :")
print(U.round(2))
print("행렬 U의 크기 : ", np.shape(U))

행렬 U :
[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]
행렬 U의 크기 :  (4, 4)


In [5]:
print("특이값 벡터 : ")
print(s.round(2))
print("특이값 벡터의 크기 : ", np.shape(s))

특이값 벡터 : 
[2.69 2.05 1.73 0.77]
특이값 벡터의 크기 :  (4,)


np.linalg.svd( )는 특이값 분해의 결과로 대각 행렬이 아니라 특이값 리스트를 반환함.  
때문에, 이것을 다시 대각 행렬로 바꿔야 함.

In [11]:
S = np.zeros((4, 9))
S[:4, :4] = np.diag(s)
print(S.round(2))

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]


In [12]:
print("직교행렬 VT : ")
print(VT.round(2))
print("직교행렬 VT의 크기 : ", np.shape(VT))

직교행렬 VT : 
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]
직교행렬 VT의 크기 :  (9, 9)


In [18]:
print(A)
B = np.dot(np.dot(U, S), VT).round(2)
print(B)
print(np.allclose(A, B))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.  0.  0.  1. -0.  1.  1.  0.  0.]
 [ 0. -0. -0.  1.  1. -0.  1. -0. -0.]
 [ 0.  1.  1. -0.  2. -0. -0.  0.  0.]
 [ 1. -0.  0.  0. -0.  0. -0.  1.  1.]]
True


### Truncated SVD  
위에서 수행한 것은 Full SVD.  
이제, 토픽의 수(t)가 2인 Truncated SVD(절단된 SVD)에 대해 알아보자.

In [19]:
# 특이값 상위 2개만 보존
S = S[:2,:2]

print('대각 행렬 S :')
print(S.round(2))

대각 행렬 S :
[[2.69 0.  ]
 [0.   2.05]]


In [20]:
U = U[:,:2]
print('행렬 U :')
print(U.round(2))

행렬 U :
[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]


In [21]:
VT = VT[:2,:]
print('직교행렬 VT :')
print(VT.round(2))

직교행렬 VT :
[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]


In [22]:
A_prime = np.dot(np.dot(U,S), VT)
print(A)
print(A_prime.round(2))

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]
[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]


Truncated SVD로 구한 U, S, VT를 곱하면 기존의 A와는 다른 결과가 나온다.  
값이 손실되었기 때문에(=차원 축소) 기존의 A행렬로 복구할 수 없는 것이다.

### 해석  

축소된 U는 4 × 2의 크기를 가지는데, 이는 잘 생각해보면 문서의 개수 × 토픽의 수 t의 크기입니다. 단어의 개수인 9는 유지되지 않는데 문서의 개수인 4의 크기가 유지되었으니 4개의 문서 각각을 2개의 값으로 표현하고 있습니다. 즉, U의 각 행은 잠재 의미를 표현하기 위한 수치화 된 각각의 문서 벡터라고 볼 수 있습니다. 축소된 VT는 2 × 9의 크기를 가지는데, 이는 잘 생각해보면 토픽의 수 t × 단어의 개수의 크기입니다. VT의 각 열은 잠재 의미를 표현하기 위해 수치화된 각각의 단어 벡터라고 볼 수 있습니다.

이 문서 벡터들과 단어 벡터들을 통해 다른 문서의 유사도, 다른 단어의 유사도, 단어(쿼리)로부터 문서의 유사도를 구하는 것들이 가능해집니다.